<a href="https://colab.research.google.com/github/subhasisj/python-projects/blob/master/python-projects/Machine%20Learning%20Projects/PySpark%20ML/sentiment_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sources [1,742

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data = spark.read.csv('/content/drive/My Drive/Colab Files/clean_tweet.csv',header=True)
data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- text: string (nullable = true)
 |-- target: string (nullable = true)



In [0]:
data.show(5)

+---+--------------------+------+
|_c0|                text|target|
+---+--------------------+------+
|  0|awww that s a bum...|     0|
|  1|is upset that he ...|     0|
|  2|i dived many time...|     0|
|  3|my whole body fee...|     0|
|  4|no it s not behav...|     0|
+---+--------------------+------+
only showing top 5 rows



In [0]:
data.count()

1600000

In [0]:
data = data.dropna()
data.count()

1596753

In [0]:
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.types import StructType,StructField,IntegerType
from pyspark.ml.feature import CountVectorizer,HashingTF,IDF,Tokenizer,StringIndexer,Imputer

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression,NaiveBayes


In [0]:
( train_set ,test_set ,validation_set ) = data.randomSplit([0.98,0.01,0.01],seed = 2000)

In [0]:
tokenizer = Tokenizer(inputCol = 'text',outputCol = 'words')

hashtf = HashingTF(numFeatures=2**16,inputCol = 'words',outputCol = 'tf')
# cv = CountVectorizer(vocabSize=2**16, inputCol="words", outputCol='cv')

idf = IDF(inputCol = 'tf', outputCol = 'features' ,minDocFreq =5)

label_string_indexer = StringIndexer(inputCol = 'target',outputCol= 'label' )

# log_reg = LogisticRegression(maxIter=100)
# nb = NaiveBayes()
pipeline = Pipeline(stages = [tokenizer,hashtf,idf,label_string_indexer])

pipelineFit = pipeline.fit(train_set)
train_df = pipelineFit.transform(train_set)
validation_set_df = pipelineFit.transform(validation_set)
train_df.show(5)

+----+--------------------+------+--------------------+--------------------+--------------------+-----+
| _c0|                text|target|               words|                  tf|            features|label|
+----+--------------------+------+--------------------+--------------------+--------------------+-----+
|   0|awww that s a bum...|     0|[awww, that, s, a...|(65536,[8436,8847...|(65536,[8436,8847...|  0.0|
|   1|is upset that he ...|     0|[is, upset, that,...|(65536,[1444,2071...|(65536,[1444,2071...|  0.0|
|  10|spring break in p...|     0|[spring, break, i...|(65536,[20639,207...|(65536,[20639,207...|  0.0|
| 100|body of missing n...|     0|[body, of, missin...|(65536,[9639,1302...|(65536,[9639,1302...|  0.0|
|1000|tulip um that wou...|     0|[tulip, um, that,...|(65536,[3331,8436...|(65536,[3331,8436...|  0.0|
+----+--------------------+------+--------------------+--------------------+--------------------+-----+
only showing top 5 rows



In [0]:
log_reg = LogisticRegression(maxIter=100)
log_reg_model=log_reg.fit(train_df)
preds = log_reg_model.transform(validation_set_df)

In [0]:
preds.show(5)

+------+--------------------+------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|   _c0|                text|target|               words|                  tf|            features|label|       rawPrediction|         probability|prediction|
+------+--------------------+------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|100186|very confused bou...|     0|[very, confused, ...|(65536,[81,4492,9...|(65536,[81,4492,9...|  0.0|[-1.5765048259390...|[0.17129105361402...|       1.0|
|100419|i think i need to...|     0|[i, think, i, nee...|(65536,[4492,4853...|(65536,[4492,4853...|  0.0|[1.85194049762095...|[0.86435477869155...|       0.0|
|100431|ugly rainy day he...|     0|[ugly, rainy, day...|(65536,[7713,8436...|(65536,[7713,8436...|  0.0|[3.69287299938189...|[0.97570460366790...|       0.0|
|100482|i saw u perform i...|     0|[i, saw, u

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(preds)

0.860456807006464

In [0]:
accuracy = preds.filter(preds.label == preds.prediction).count() / float(validation_set.count())
accuracy

0.7885553470919324

In [0]:
evaluator.getMetricName()


'areaUnderROC'